In [ ]:
%%capture
!apt-get update
!apt-get install -y poppler-utils
!pip install easyocr[torch] opencv-python-headless pillow pdf2image poppler-utils

In [7]:
import cv2
import numpy as np
import os
from PIL import Image
import easyocr
from datetime import datetime

reader = easyocr.Reader(['vi', 'en'], gpu=False)

class SGKMarkdownGenerator:
    def __init__(self, output_dir='sgk_complete'):
        self.output_dir = output_dir
        self.images_dir = f"{output_dir}/images"
        os.makedirs(self.images_dir, exist_ok=True)
        self.image_counter = 0

    def load_image_safe(self, img_path):
        img = cv2.imread(img_path)
        return img

    def ocr_full_page(self, img):
        """OCR TOÀN BỘ trang SGK"""
        result = reader.readtext(img, detail=1)
        full_text = []

        for item in result:
            if len(item) >= 2:
                text = item[1][0] if isinstance(item[1], tuple) else item[1]
                conf = item[1][1] if isinstance(item[1], tuple) else 0.9
                if conf > 0.5 and len(text.strip()) > 1:
                    full_text.append(text.strip())

        return ' '.join(full_text)

    def detect_illustrations(self, img):
      h, w = img.shape[:2]
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

      # Tăng khả năng gom block
      blurred = cv2.GaussianBlur(gray, (5, 5), 0)
      edges = cv2.Canny(blurred, 30, 120)

      kernel = np.ones((9,9), np.uint8)
      edges = cv2.dilate(edges, kernel, iterations=2)

      contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      illustrations = []

      for cnt in contours:
          x, y, cw, ch = cv2.boundingRect(cnt)
          area = cw * ch
          aspect = cw / ch

          if (
              cw > w * 0.25 and
              ch > h * 0.12 and
              area > w * h * 0.035 and
              0.5 < aspect < 4.5
          ):
              roi = img[y:y+ch, x:x+cw]
              std = np.std(cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY))
              if std > 12:
                  illustrations.append({
                      'bbox': (x, y, cw, ch),
                      'roi': roi
                  })

      illustrations.sort(key=lambda x: (x['bbox'][1], x['bbox'][0]))
      return illustrations



    def save_image(self, roi, prefix, page_num, idx):
        img_name = f"{prefix}_page{page_num}_{idx:02d}.jpg"
        img_path = f"{self.images_dir}/{img_name}"

        h, w = roi.shape[:2]
        if max(h, w) > 800:
            scale = 800 / max(h, w)
            roi = cv2.resize(roi, (int(w*scale), int(h*scale)))

        cv2.imwrite(img_path, roi)
        self.image_counter += 1
        return img_name

    def get_caption_text(self, img, bbox):
        x, y, w, h = bbox

        caption_start_y = y + h
        caption_end_y = min(caption_start_y + 80, img.shape[0])
        caption_start_x = max(0, x - 50)
        caption_end_x = min(x + w + 50, img.shape[1])

        if caption_start_y < img.shape[0]:
            caption_roi = img[caption_start_y:caption_end_y, caption_start_x:caption_end_x]
            return self.ocr_full_page(caption_roi)
        return ""

    def generate_full_markdown(self, img, illustrations, page_num):
        md_content = []

        # Header
        md_content.append(f"# SGK Hóa Học - Trang {page_num}")
        md_content.append("")

        # Thumbnail trang gốc
        thumb_name = self.save_image(img, "thumb", page_num, 0)
        # md_content.append(f"![Trang {page_num}](images/{thumb_name})")
        md_content.append("")

        # FULL TEXT OCR
        full_text = self.ocr_full_page(img)
        md_content.append("")
        md_content.append(full_text)
        md_content.append("")
        md_content.append("---")
        md_content.append("")

        if illustrations:
            md_content.append("## Hình minh họa")
            md_content.append("")

            for i, illust in enumerate(illustrations, 1):
                img_name = self.save_image(illust['roi'], "hinh", page_num, i)
                caption = self.get_caption_text(img, illust['bbox'])

                md_content.append(f"### Hình {i}")
                md_content.append(f"![Hình {i}](images/{img_name})")

                if caption.strip():
                    md_content.append(f"**Chú thích**: {caption.strip()}")
                md_content.append("")
                md_content.append("---")
                md_content.append("")

        md_content.append("##Text blocks chi tiết")
        md_content.append("")

        return md_content

    def process_page(self, img_path, page_num=37):
        """Xử lý hoàn chỉnh 1 trang SGK"""
        img = self.load_image_safe(img_path)
        if img is None:
            return None

        illustrations = self.detect_illustrations(img)
        print(f"Tìm thấy {len(illustrations)} hình minh họa")

        md_content = self.generate_full_markdown(img, illustrations, page_num)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M")
        md_filename = f"sgk_page_{page_num}_{timestamp}.md"
        md_path = f"{self.output_dir}/{md_filename}"

        with open(md_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(md_content))

        return {
            'markdown': md_path,
            'images_dir': self.images_dir,
            'total_images': self.image_counter
        }

generator = SGKMarkdownGenerator('sgk_h2so4_final1')
result = generator.process_page('/content/img_39.jpg', page_num=37)



Tìm thấy 2 hình minh họa


In [10]:
from pdf2image import convert_from_path
import os

def pdf_to_images(pdf_path, out_dir="pages", dpi=200):
    os.makedirs(out_dir, exist_ok=True)
    images = convert_from_path(pdf_path, dpi=dpi)
    paths = []
    for i, img in enumerate(images, 1):
        path = f"{out_dir}/page_{i:03d}.jpg"
        img.save(path, "JPEG")
        paths.append(path)
    return paths


In [11]:
def process_book(pdf_path, output_root="sgk_output"):
    page_imgs = pdf_to_images(pdf_path, f"{output_root}/pages")
    generator = SGKMarkdownGenerator(output_root)

    results = []
    for i, img_path in enumerate(page_imgs, 1):
        print(f"Đang xử lý trang {i}")
        res = generator.process_page(img_path, page_num=i)
        results.append(res)

    return results


In [19]:
from pdf2image import convert_from_path
import os

def pdf_to_images(pdf_path, out_dir="pages", dpi=200):
    os.makedirs(out_dir, exist_ok=True)
    images = convert_from_path(
        pdf_path,
        dpi=dpi,
        poppler_path="/usr/bin"
    )
    paths = []
    for i, img in enumerate(images, 1):
        path = f"{out_dir}/page_{i:03d}.jpg"
        img.save(path, "JPEG")
        paths.append(path)
    return paths


In [20]:
results = process_book("/content/khtn-8-31trang.pdf", output_root="sgk_hoa10")


Đang xử lý trang 1
Tìm thấy 5 hình minh họa
Đang xử lý trang 2
Tìm thấy 0 hình minh họa
Đang xử lý trang 3
Tìm thấy 2 hình minh họa
Đang xử lý trang 4
Tìm thấy 1 hình minh họa
Đang xử lý trang 5
Tìm thấy 1 hình minh họa
Đang xử lý trang 6
Tìm thấy 0 hình minh họa
Đang xử lý trang 7
Tìm thấy 0 hình minh họa
Đang xử lý trang 8
Tìm thấy 1 hình minh họa
Đang xử lý trang 9
Tìm thấy 0 hình minh họa
Đang xử lý trang 10
Tìm thấy 2 hình minh họa
Đang xử lý trang 11
Tìm thấy 2 hình minh họa
Đang xử lý trang 12
Tìm thấy 1 hình minh họa
Đang xử lý trang 13
Tìm thấy 2 hình minh họa
Đang xử lý trang 14
Tìm thấy 1 hình minh họa
Đang xử lý trang 15
Tìm thấy 2 hình minh họa
Đang xử lý trang 16
Tìm thấy 3 hình minh họa
Đang xử lý trang 17
Tìm thấy 2 hình minh họa
Đang xử lý trang 18
Tìm thấy 1 hình minh họa
Đang xử lý trang 19
Tìm thấy 4 hình minh họa
Đang xử lý trang 20
Tìm thấy 2 hình minh họa
Đang xử lý trang 21
Tìm thấy 1 hình minh họa
Đang xử lý trang 22
Tìm thấy 1 hình minh họa
Đang xử lý trang 23

In [18]:
import shutil

shutil.make_archive("myfolder", "zip", "/content/sgk_hoa10")
from google.colab import files
files.download("myfolder.zip")
